## Option 1: BBC 21st Century's greatest films

Last year the BBC polled 177 film critics to get their picks for the best films of the century so far. While the BBC's aggregate pole is interesting, the long list include everyone who voted is perhaps more revealing from the data standpoint:

http://www.bbc.com/culture/story/20160819-the-21st-centurys-100-greatest-films-who-voted

Our goal for this project would be to scrape this page -- using beautiful soup and regular expressions -- in order to make a searchable database that would allow us to investigate all of the films listed by all of the critics.

If you choose this project, the main challenge will be to come up with a database schema that organizes information by film as well as by critic. 

From a geocoding standpoint we would visualize this data set based on the country of the critics, and we could, with extra research, View the data set buy the country of the filmmaker.

From an exploratory standpoint, here are some questions we could ask: 

1. Which countries have the most directors?
2. Which directors have the most movies selected?
3. What year had the most movies be selected?
4. What are some other questions you can think of?

The overall challenge, once the initial page is properly formatted into a database, is to find a way to enhance this data set with another source, and integrate that information with the information from the BBC poll.

## Option 2: Supreme Court Arguments
One group project I am proposing, would be to create a database of arguments before the US Supreme Court. The Supreme Court makes transcriptions of all arguments available on its website:

https://www.supremecourt.gov/oral_arguments/argument_transcript.aspx

These arguments are available as PDFs. The downloading and conversion of PDFs to text is neither trivial nor particularly interesting of a challenge. The real challenge is what to do with the data once we have the text. 

Below, I show the code that I used to download all the PDFs of the the oral arguments before the Supreme Court from 2016, which I then convert to text files: that is where the fun starts. The goal of this project Will be to use regular expressions to parse the text to do oral arguments, that we can then search and measure the word spoken by the Supreme Court justices from case to case and across cases.

The end goal is open: we want to have well structured data with the words of the justices entered. We also want to have a useful data set for each case that includes further information such as the final decision, the votes of each justice, who wrote the decision (and perhaps the dissenting opinion too). It is up to you what should ultimately be included and how we will be able to search through the data and view results.

From an exploratory standpoint, here are some questions we could ask: 

1. Which justice speaks the most from case to case?
2. What are the most frequent words used by each justice across cases?
3. Does frequency of speech during arguments have any relationship toThe decision?
4. What are some other questions you can think of?

As far as geocoding goes: one way we could arrange this data on the map would be by finding the state where each of these cases originated. I have not found the best source for this, so it is an open question--part of this project will entail researching data sets.


I begin by scraping the links to all the transcriptions using **beautiful soup**--if you were to choose to do this project, give it also want to speak this page for the rest of its information such as the name of the case, the docket number, etc..

## 0. Scraping


### 0-1. BBC crtics

In [1]:
# BBC 21st Century's greatest films


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [3]:
BBCfilms_html = "http://www.bbc.com/culture/story/20160819-the-21st-centurys-100-greatest-films-who-voted"
raw_BBCfilms_html = urlopen(BBCfilms_html).read()
soup_doc_BBCfilms = BeautifulSoup(raw_BBCfilms_html, "html.parser")
print(type(soup_doc_BBCfilms))


<class 'bs4.BeautifulSoup'>


In [4]:
# soup_doc_BBCfilms


In [5]:
the_critics = soup_doc_BBCfilms.find_all('div', attrs={'class': 'body-content'})
# the_critics


In [6]:
import re


In [7]:
type(the_critics)
len(the_critics)   # 1
# the_critics[0]



1

In [8]:
the_critics_p = the_critics[0].find_all('p')[1:]
type(the_critics_p)
the_critics_p[1]


<p>1. Mulholland Drive (David Lynch, 2001)<br/>2. In the Mood for Love (Wong Kar-wai, 2000)<br/>3. The Tree of Life (Terrence Malick, 2011)<br/>4. Yi Yi: A One and a Two (Edward Yang, 2000)<br/>5. Goodbye to Language (Jean-Luc Godard, 2014)<br/>6. The White Meadows (Mohammad Rasoulof, 2009)<br/>7. Night Across the Street (Raoul Ruiz, 2012)<br/>8. Certified Copy (Abbas Kiarostami, 2010)<br/>9. Sparrow (Johnnie To, 2008)<br/>10. Fados (Carlos Saura, 2007)</p>

In [9]:
all_lines = the_critics[0](text=True)[2:-15]


In [10]:
BBC_critics = []
for i in range(0, int(len(all_lines)/11) ):
    critic_info = re.search(r"^(.*)–(.*)\((.*)\)$", all_lines[i*11])
    film_info = all_lines[i*11+1:i*11+11]
#     print(type(film_info))
    films_dict = []
    for info in film_info:
        movie_info = re.search(r"^.*\.(.*)\((.*)\,(.*)\)$", info)
#         print(movie_info.group(0))
#         print(movie_info.group(1))
#         print(movie_info.group(2))
#         print(movie_info.group(3))
#         break
        films_info = {
            "movie_title"   : movie_info.group(1).strip(),
            "movie_director": movie_info.group(2).strip(),
            "movie_year"    : movie_info.group(3).strip()
        }
        films_dict.append(films_info)
    one_critic = {
        "critic_name"       : critic_info.group(1).strip(),
        "critic_affiliation": critic_info.group(2).strip(),
        "critic_country"    : critic_info.group(3),
        "films"             : films_dict
    }
    BBC_critics.append(one_critic)
#     break
BBC_critics


[{'critic_affiliation': 'Freelance film critic',
  'critic_country': 'US',
  'critic_name': 'Simon Abrams',
  'films': [{'movie_director': 'David Lynch',
    'movie_title': 'Mulholland Drive',
    'movie_year': '2001'},
   {'movie_director': 'Wong Kar-wai',
    'movie_title': 'In the Mood for Love',
    'movie_year': '2000'},
   {'movie_director': 'Terrence Malick',
    'movie_title': 'The Tree of Life',
    'movie_year': '2011'},
   {'movie_director': 'Edward Yang',
    'movie_title': 'Yi Yi: A One and a Two',
    'movie_year': '2000'},
   {'movie_director': 'Jean-Luc Godard',
    'movie_title': 'Goodbye to Language',
    'movie_year': '2014'},
   {'movie_director': 'Mohammad Rasoulof',
    'movie_title': 'The White Meadows',
    'movie_year': '2009'},
   {'movie_director': 'Raoul Ruiz',
    'movie_title': 'Night Across the Street',
    'movie_year': '2012'},
   {'movie_director': 'Abbas Kiarostami',
    'movie_title': 'Certified Copy',
    'movie_year': '2010'},
   {'movie_director':

In [11]:
len(BBC_critics)

177

In [12]:
import pandas as pd


In [13]:
BBC_critics_df = pd.DataFrame(BBC_critics)
BBC_critics_df.head()


critic_affiliation critic_country       critic_name  \
0  Freelance film critic             US      Simon Abrams   
1  Freelance film critic             US         Sam Adams   
2  Freelance film critic             US      Thelma Adams   
3   Rolling Stone Mexico         Mexico    Arturo Aguilar   
4            BBC Culture             UK  Matthew Anderson   

                                               films  
0  [{'movie_title': 'Mulholland Drive', 'movie_di...  
1  [{'movie_title': 'In the Mood for Love', 'movi...  
2  [{'movie_title': 'Zero Dark Thirty', 'movie_di...  
3  [{'movie_title': 'In the Mood for Love', 'movi...  
4  [{'movie_title': 'The Piano Teacher', 'movie_d...

In [ ]:
# a few more dataframes
# - films (remove duplicates)
# - directors


### 0-2. Films

In [13]:
films_dict = []


In [14]:
for critic in BBC_critics:
    films_dict = films_dict + critic['films']
films_dict

[{'movie_director': 'David Lynch',
  'movie_title': 'Mulholland Drive',
  'movie_year': '2001'},
 {'movie_director': 'Wong Kar-wai',
  'movie_title': 'In the Mood for Love',
  'movie_year': '2000'},
 {'movie_director': 'Terrence Malick',
  'movie_title': 'The Tree of Life',
  'movie_year': '2011'},
 {'movie_director': 'Edward Yang',
  'movie_title': 'Yi Yi: A One and a Two',
  'movie_year': '2000'},
 {'movie_director': 'Jean-Luc Godard',
  'movie_title': 'Goodbye to Language',
  'movie_year': '2014'},
 {'movie_director': 'Mohammad Rasoulof',
  'movie_title': 'The White Meadows',
  'movie_year': '2009'},
 {'movie_director': 'Raoul Ruiz',
  'movie_title': 'Night Across the Street',
  'movie_year': '2012'},
 {'movie_director': 'Abbas Kiarostami',
  'movie_title': 'Certified Copy',
  'movie_year': '2010'},
 {'movie_director': 'Johnnie To',
  'movie_title': 'Sparrow',
  'movie_year': '2008'},
 {'movie_director': 'Carlos Saura',
  'movie_title': 'Fados',
  'movie_year': '2007'},
 {'movie_dir

In [15]:
films_df = pd.DataFrame(films_dict)
films_df.head()


movie_director             movie_title movie_year
0      David Lynch        Mulholland Drive       2001
1     Wong Kar-wai    In the Mood for Love       2000
2  Terrence Malick        The Tree of Life       2011
3      Edward Yang  Yi Yi: A One and a Two       2000
4  Jean-Luc Godard     Goodbye to Language       2014

In [16]:
films_df.shape


(1770, 3)

In [18]:
films_df_unique = films_df.drop_duplicates(['movie_title'])
# films_df['movie_title'].value_counts()
films_df_unique.shape


(597, 3)

### 0-3. directors

In [41]:
directors = list(set(films_df_unique['movie_director']))

In [42]:
len(directors)

408

In [71]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("- - icognito")

driver = webdriver.Chrome(chrome_options = chrome_options)
driver.get("http://www.imdb.com/")


In [69]:
def get_directors_birthplaces(directors):
    directors_dict = []
    
    for director_name in directors:
        director = {
            'name' : director_name,
        }
        
        search_box = driver.find_element_by_xpath('//*[@id="navbar-query"]')
        search_box.clear()
        search_box.send_keys(director_name)
        search_button = driver.find_element_by_xpath('//*[@id="navbar-submit-button"]/div')
        search_button.click()
        print(director_name)

        page_xpath = driver.find_element_by_xpath('//*[@id="main"]/div')

        page_doc = BeautifulSoup(driver.page_source, 'html.parser')
        name_info = page_doc.find_all('h3')

        for a_list in name_info:
            if "Names" in a_list.text:
                first = a_list.find_next('tr')
                next_name = first.find_next('a')
                url = "http://www.imdb.com"+next_name['href']
                print(url)
                driver.get(url)

                birth_place = driver.find_element_by_xpath('//*[@id="name-born-info"]/a')
                director.update( {
                    'birthplace' : birth_place.text
                } )
                break

        print(director)
        directors_dict.append(director)
    return directors_dict


In [49]:
directors_dict1 = get_directors_birthplaces(directors[:50])
directors_dict1

Werner Herzog
http://www.imdb.com/name/nm0001348/?ref_=fn_al_nm_1
{'name': 'Werner Herzog', 'birthplace': 'Munich, Bavaria, Germany'}
Debra Granik
http://www.imdb.com/name/nm0335138/?ref_=fn_al_nm_1
{'name': 'Debra Granik', 'birthplace': 'Cambridge, Massachusetts, USA'}
David Michôd
http://www.imdb.com/name/nm2391575/?ref_=fn_al_nm_1
{'name': 'David Michôd'}
Michael Mann
http://www.imdb.com/name/nm0000520/?ref_=fn_al_nm_1
{'name': 'Michael Mann', 'birthplace': 'Chicago, Illinois, USA'}
Wang Bing
http://www.imdb.com/name/nm7432856/?ref_=fn_al_nm_1
{'name': 'Wang Bing'}
Nicolas Winding Refn
http://www.imdb.com/name/nm0716347/?ref_=fn_al_nm_1
{'name': 'Nicolas Winding Refn', 'birthplace': 'Copenhagen, Denmark'}
Alfonso Cuarón
http://www.imdb.com/name/nm0190859/?ref_=fn_al_nm_1
{'name': 'Alfonso Cuarón', 'birthplace': 'Mexico City, Distrito Federal, Mexico'}
Rajat Kapoor
http://www.imdb.com/name/nm0438494/?ref_=fn_al_nm_1
{'name': 'Rajat Kapoor', 'birthplace': 'New Delhi, India'}
Kelly Rei

[{'birthplace': 'Munich, Bavaria, Germany', 'name': 'Werner Herzog'},
 {'birthplace': 'Cambridge, Massachusetts, USA', 'name': 'Debra Granik'},
 {'name': 'David Michôd'},
 {'birthplace': 'Chicago, Illinois, USA', 'name': 'Michael Mann'},
 {'name': 'Wang Bing'},
 {'birthplace': 'Copenhagen, Denmark', 'name': 'Nicolas Winding Refn'},
 {'birthplace': 'Mexico City, Distrito Federal, Mexico',
  'name': 'Alfonso Cuarón'},
 {'birthplace': 'New Delhi, India', 'name': 'Rajat Kapoor'},
 {'name': 'Kelly Reichardt'},
 {'birthplace': 'Athens, Greece', 'name': 'Yorgos Lanthimos'},
 {'birthplace': 'Denver, Colorado, USA', 'name': 'Travis Wilkerson'},
 {'birthplace': 'Paris, France', 'name': 'Mia Hansen-Løve'},
 {'birthplace': 'Ferrara, Emilia-Romagna, Italy',
  'name': 'Michelangelo Antonioni'},
 {'birthplace': 'Paris, France', 'name': 'Roman Polanski'},
 {'birthplace': 'Sapporo, Hokkaido, Japan', 'name': 'Satoshi Kon'},
 {'birthplace': 'Oakland, California, USA', 'name': 'Cary Joji Fukunaga'},
 {'na

In [50]:
directors_dict2 = get_directors_birthplaces(directors[50:100])


Bart Layton
http://www.imdb.com/name/nm1717925/?ref_=fn_al_nm_1
{'name': 'Bart Layton'}
Wong Kar-wai
http://www.imdb.com/name/nm0939182/?ref_=fn_al_nm_1
{'name': 'Wong Kar-wai', 'birthplace': 'Shanghai, China'}
Evan Goldberg and Seth Rogen
{'name': 'Evan Goldberg and Seth Rogen'}
Rian Johnson
http://www.imdb.com/name/nm0426059/?ref_=fn_al_nm_1
{'name': 'Rian Johnson', 'birthplace': 'Silver Spring, Maryland, USA'}
Andrew Dosunmu
http://www.imdb.com/name/nm0234506/?ref_=fn_al_nm_1
{'name': 'Andrew Dosunmu'}
Christopher Morris
http://www.imdb.com/name/nm0606439/?ref_=fn_al_nm_1
{'name': 'Christopher Morris', 'birthplace': 'Bristol, England, UK'}
Hong Sang-soo
http://www.imdb.com/name/nm0393254/?ref_=fn_al_nm_1
{'name': 'Hong Sang-soo', 'birthplace': 'Seoul, Korea'}
Benh Zeitlin
http://www.imdb.com/name/nm1022455/?ref_=fn_al_nm_1
{'name': 'Benh Zeitlin', 'birthplace': 'Manhattan, New York, USA'}
Nikolaus Geyrhalter
http://www.imdb.com/name/nm0315430/?ref_=fn_al_nm_1
{'name': 'Nikolaus Geyr

In [51]:
directors_dict3 = get_directors_birthplaces(directors[100:150])


Ulrich Seidl
http://www.imdb.com/name/nm0782430/?ref_=fn_al_nm_1
{'name': 'Ulrich Seidl', 'birthplace': 'Vienna, Austria'}
Sara Fattahi
http://www.imdb.com/name/nm7986571/?ref_=fn_al_nm_1
{'name': 'Sara Fattahi'}
Elia Suleiman
http://www.imdb.com/name/nm0837839/?ref_=fn_al_nm_1
{'name': 'Elia Suleiman', 'birthplace': 'Nazareth, Israel'}
Valeska Grisebach
http://www.imdb.com/name/nm1007136/?ref_=fn_al_nm_1
{'name': 'Valeska Grisebach', 'birthplace': 'Bremen, West Germany [now Germany]'}
James Marsch
http://www.imdb.com/name/nm3193396/?ref_=fn_al_nm_1
{'name': 'James Marsch'}
Whit Stillman
http://www.imdb.com/name/nm0001775/?ref_=fn_al_nm_1
{'name': 'Whit Stillman', 'birthplace': 'Washington, District of Columbia, USA'}
Derek Cianfrance
http://www.imdb.com/name/nm0161834/?ref_=fn_al_nm_1
{'name': 'Derek Cianfrance', 'birthplace': 'Lakewood, Colorado, USA'}
Manoel de Oliveira
http://www.imdb.com/name/nm0210701/?ref_=fn_al_nm_1
{'name': 'Manoel de Oliveira', 'birthplace': 'Oporto, Portugal

In [72]:
directors_dict4 = get_directors_birthplaces(directors[150:200])


Michel Hazanavicius
http://www.imdb.com/name/nm0371890/?ref_=fn_al_nm_1
{'name': 'Michel Hazanavicius', 'birthplace': 'Paris, France'}
Adam McKay
http://www.imdb.com/name/nm0570912/?ref_=fn_al_nm_1
{'name': 'Adam McKay', 'birthplace': 'Philadelphia, Pennsylvania, USA'}
Sion Sono
http://www.imdb.com/name/nm0814469/?ref_=fn_al_nm_1
{'name': 'Sion Sono', 'birthplace': 'Toyokawa, Aichi, Japan'}
Ken Loach
http://www.imdb.com/name/nm0516360/?ref_=fn_al_nm_1
{'name': 'Ken Loach', 'birthplace': 'Nuneaton, Warwickshire, England, UK'}
Lee Chang-dong
http://www.imdb.com/name/nm0496969/?ref_=fn_al_nm_1
{'name': 'Lee Chang-dong', 'birthplace': 'Daegu, South Korea'}
Clio Barnard
http://www.imdb.com/name/nm1163237/?ref_=fn_al_nm_1
{'name': 'Clio Barnard'}
Pacho Velez and Stephanie Spray
{'name': 'Pacho Velez and Stephanie Spray'}
Eran Kolirin
http://www.imdb.com/name/nm0464115/?ref_=fn_al_nm_1
{'name': 'Eran Kolirin'}
Stephen Frears
http://www.imdb.com/name/nm0001241/?ref_=fn_al_nm_1
{'name': 'Stephe

In [74]:
directors_dict5 = get_directors_birthplaces(directors[200:250])

Gareth Edwards
http://www.imdb.com/name/nm2284484/?ref_=fn_al_nm_1
{'name': 'Gareth Edwards'}
Peter Mullan
http://www.imdb.com/name/nm0611932/?ref_=fn_al_nm_1
{'name': 'Peter Mullan', 'birthplace': 'Peterhead, Scotland, UK'}
Denis Villeneuve
http://www.imdb.com/name/nm0898288/?ref_=fn_al_nm_1
{'name': 'Denis Villeneuve', 'birthplace': 'Trois-Rivières, Québec, Canada'}
John Carney
http://www.imdb.com/name/nm0138809/?ref_=fn_al_nm_1
{'name': 'John Carney', 'birthplace': 'Dublin, Ireland'}
Albert Serra
http://www.imdb.com/name/nm2247200/?ref_=fn_al_nm_1
{'name': 'Albert Serra', 'birthplace': 'Banyoles, Girona, Catalonia, Spain'}
James Cameron
http://www.imdb.com/name/nm0000116/?ref_=fn_al_nm_1
{'name': 'James Cameron', 'birthplace': 'Kapuskasing, Ontario, Canada'}
Gus Van Sant
http://www.imdb.com/name/nm0001814/?ref_=fn_al_nm_1
{'name': 'Gus Van Sant', 'birthplace': 'Louisville, Kentucky, USA'}
László Nemes
http://www.imdb.com/name/nm1841577/?ref_=fn_al_nm_1
{'name': 'László Nemes', 'birt

In [76]:
directors_dict6 = get_directors_birthplaces(directors[250:300])

Nouri Bouzid
http://www.imdb.com/name/nm0100525/?ref_=fn_al_nm_1
{'name': 'Nouri Bouzid'}
Lynne Ramsay
http://www.imdb.com/name/nm0708903/?ref_=fn_al_nm_1
{'name': 'Lynne Ramsay', 'birthplace': 'Glasgow, Strathclyde, Scotland, UK'}
Hayao Miyazaki
http://www.imdb.com/name/nm0594503/?ref_=fn_al_nm_1
{'name': 'Hayao Miyazaki', 'birthplace': 'Tokyo, Japan'}
Miguel Gomes
http://www.imdb.com/name/nm0326937/?ref_=fn_al_nm_1
{'name': 'Miguel Gomes', 'birthplace': 'Lisbon, Portugal'}
Baz Luhrmann
http://www.imdb.com/name/nm0525303/?ref_=fn_al_nm_1
{'name': 'Baz Luhrmann', 'birthplace': 'New South Wales, Australia'}
Vishal Bhardwaj
http://www.imdb.com/name/nm0080235/?ref_=fn_al_nm_1
{'name': 'Vishal Bhardwaj'}
David Lynch
http://www.imdb.com/name/nm0000186/?ref_=fn_al_nm_1
{'name': 'David Lynch', 'birthplace': 'Missoula, Montana, USA'}
Joe Cornish
http://www.imdb.com/name/nm0180428/?ref_=fn_al_nm_1
{'name': 'Joe Cornish', 'birthplace': 'London, England, UK'}
Carlos Reygadas
http://www.imdb.com/n

In [78]:
directors_dict7 = get_directors_birthplaces(directors[300:350])

Kim Ki-duk
http://www.imdb.com/name/nm1104118/?ref_=fn_al_nm_1
{'name': 'Kim Ki-duk', 'birthplace': 'Bonghwa, South Korea'}
Cristian Mungiu
http://www.imdb.com/name/nm0612816/?ref_=fn_al_nm_1
{'name': 'Cristian Mungiu', 'birthplace': 'Iasi, Romania'}
Thom Andersen
http://www.imdb.com/name/nm0026263/?ref_=fn_al_nm_1
{'name': 'Thom Andersen', 'birthplace': 'Chicago, Illinois, USA'}
Jason Reitman
http://www.imdb.com/name/nm0718646/?ref_=fn_al_nm_1
{'name': 'Jason Reitman', 'birthplace': 'Montreal, Quebec, Canada'}
George Miller
http://www.imdb.com/name/nm0004306/?ref_=fn_al_nm_1
{'name': 'George Miller', 'birthplace': 'Chinchilla, Queensland, Australia'}
Eran Riklis
http://www.imdb.com/name/nm0726954/?ref_=fn_al_nm_1
{'name': 'Eran Riklis', 'birthplace': 'Beersheba, Israel'}
Hany Abu-Assad
http://www.imdb.com/name/nm0009463/?ref_=fn_al_nm_1
{'name': 'Hany Abu-Assad', 'birthplace': 'Nazareth, Israel'}
Michelangelo Frammartino
http://www.imdb.com/name/nm1506005/?ref_=fn_al_nm_1
{'name': 'Mi

In [80]:
directors_dict8 = get_directors_birthplaces(directors[350:400])

Marco De Angelis and Antonio Di Trapani
{'name': 'Marco De Angelis and Antonio Di Trapani'}
Gurinder Chadha
http://www.imdb.com/name/nm0149446/?ref_=fn_al_nm_1
{'name': 'Gurinder Chadha', 'birthplace': 'Nairobi, Kenya'}
Terrence Malick
http://www.imdb.com/name/nm0000517/?ref_=fn_al_nm_1
{'name': 'Terrence Malick', 'birthplace': 'Ottawa, Illinois, USA'}
Jean-Luc Godard
http://www.imdb.com/name/nm0000419/?ref_=fn_al_nm_1
{'name': 'Jean-Luc Godard', 'birthplace': 'Paris, France'}
David O. Russell
http://www.imdb.com/name/nm0751102/?ref_=fn_al_nm_1
{'name': 'David O. Russell', 'birthplace': 'New York City, New York, USA'}
Francis Ford Coppola
http://www.imdb.com/name/nm0000338/?ref_=fn_al_nm_1
{'name': 'Francis Ford Coppola', 'birthplace': 'Detroit, Michigan, USA'}
Cristi Puiu
http://www.imdb.com/name/nm1090998/?ref_=fn_al_nm_1
{'name': 'Cristi Puiu', 'birthplace': 'Bucharest, Romania'}
Zhang Yimou
http://www.imdb.com/name/nm0955443/?ref_=fn_al_nm_1
{'name': 'Zhang Yimou', 'birthplace': "X

In [82]:
directors_dict9 = get_directors_birthplaces(directors[400:])

Naji Abou Nowar
http://www.imdb.com/name/nm3578681/?ref_=fn_al_nm_1
{'name': 'Naji Abou Nowar'}
Bong Joon-ho
http://www.imdb.com/name/nm0094435/?ref_=fn_al_nm_1
{'name': 'Bong Joon-ho', 'birthplace': 'Daegu, South Korea'}
Woody Allen
http://www.imdb.com/name/nm0000095/?ref_=fn_al_nm_1
{'name': 'Woody Allen', 'birthplace': 'Brooklyn, New York City, New York, USA'}
Jacques Audiard
http://www.imdb.com/name/nm0002191/?ref_=fn_al_nm_1
{'name': 'Jacques Audiard', 'birthplace': 'Paris, France'}
Mahamat-Saleh Haroun
http://www.imdb.com/name/nm0757958/?ref_=fn_al_nm_1
{'name': 'Mahamat-Saleh Haroun', 'birthplace': 'Abéché, Chad'}
Sean Baker
http://www.imdb.com/name/nm0048918/?ref_=fn_al_nm_1
{'name': 'Sean Baker'}
Raam Reddy
http://www.imdb.com/name/nm5339365/?ref_=fn_al_nm_1
{'name': 'Raam Reddy'}
Eytan Fox
http://www.imdb.com/name/nm0297202/?ref_=fn_al_nm_1
{'name': 'Eytan Fox', 'birthplace': 'New York City, New York, USA'}


In [83]:
directors_all = directors_dict1 + directors_dict2 + directors_dict3 + directors_dict4 + directors_dict5 + directors_dict6 + directors_dict7 + directors_dict8 + directors_dict9
print(len(directors_all))

408


## 1. Which countries have the most directors?

In [85]:
directors_all_df = pd.DataFrame(directors_all)
directors_all_df.head()


birthplace           name
0       Munich, Bavaria, Germany  Werner Herzog
1  Cambridge, Massachusetts, USA   Debra Granik
2                            NaN   David Michôd
3         Chicago, Illinois, USA   Michael Mann
4                            NaN      Wang Bing

In [110]:
directors_all_df['country'] = directors_all_df['birthplace'].str.extract(r".*\,.*\,(.*)$", expand=False)
directors_all_df.head()


birthplace           name   country
0       Munich, Bavaria, Germany  Werner Herzog   Germany
1  Cambridge, Massachusetts, USA   Debra Granik       USA
2                            NaN   David Michôd       NaN
3         Chicago, Illinois, USA   Michael Mann       USA
4                            NaN      Wang Bing       NaN

In [112]:
directors_all_df['country'].value_counts().head()


 USA       99
 UK        33
 France    15
 Italy     11
 Canada     9
Name: country, dtype: int64

## 2. Which directors have the most movies selected?

In [95]:
films_df.groupby('movie_director')['movie_title'].value_counts()
# by the directors' movies


movie_director                             movie_title                                
Abbas Kiarostami                           Certified Copy                                  9
                                           Ten                                             4
                                           Like Someone In Love                            2
Abdellatif Kechiche                        Blue Is the Warmest Color                       7
Abderrahmane Sissako                       Timbuktu                                        9
                                           Waiting for Happiness                           2
Adam Curtis                                The Century of the Self                         1
Adam McKay                                 Anchorman: The Legend of Ron Burgundy           1
                                           Step Brothers                                   1
                                           Talladega Nights: The Ballad of R

In [99]:
films_df['movie_director'].value_counts().head()
# by directors


Joel and Ethan Coen     52
Paul Thomas Anderson    52
Wong Kar-wai            51
David Lynch             48
Richard Linklater       39
Name: movie_director, dtype: int64

## 3. What year had the most movies be selected?

In [98]:
films_df_unique['movie_year'].value_counts().head()


2002    44
2014    42
2010    41
2000    41
2012    40
Name: movie_year, dtype: int64

## 4. Which countries have the most critics?

In [100]:
BBC_critics_df['critic_country'].value_counts().head()


US        82
UK        18
India      5
Cuba       5
Canada     5
Name: critic_country, dtype: int64

## 5. Which affiliations have the most critics?

In [102]:
BBC_critics_df['critic_affiliation'].value_counts().head()


Freelance film critic    26
BBC Culture               7
Variety                   5
Freelance                 3
IndieWire                 3
Name: critic_affiliation, dtype: int64

# For next time
Next I used the **requests** library (which I installed via pip) to download all of the PDFs to a folder on my computer.

In [ ]:
# import requests

# for urls in all_2016_pdfs:
#     link = 'https://www.supremecourt.gov/oral_arguments/' + urls
#     book_name = "/Users/Jon/Documents/columbia_syllabus/pdf/" + link.split('/')[-1]
#     with open(book_name, 'wb') as book:
#         a = requests.get(link, stream=True)

#         for block in a.iter_content(512):
#             if not block:
#                 break

#             book.write(block)

In [ ]:
#Here I make a list of the names of the PDFs
pdf_names = [url.split('/')[-1] for url in all_2016_pdfs]


Here I use the built in **os** library to run command line actions from Python. I am using the command line based **xpdf** tool (specifically its **pdftotext** command)that converts PDF to text in a way that's faster and simpler than using Python libraries that deal with PDFs. (This is certainly not the only way to do this!)

In [ ]:
# import os
# def pdf_to_text(name):
#     folder = "/Users/Jon/Documents/columbia_syllabus/pdf/"
#     input1 = folder + name
#     txt_name = name.replace(".pdf",".txt")
#     output1 = folder + txt_name
#     os.system("pdftotext '%s' '%s'" % (input1, output1))

#Here's an example of a single command    
#os.system('pdftotext /Users/Jon/Documents/columbia_syllabus/pdf/16-605_2dp3.pdf /Users/Jon/Documents/columbia_syllabus/pdf/16-605_2dp3.txt')


In [ ]:
# #This those to the names and sends them to the function
# for pdf_file in pdf_names:
#     pdf_to_text(pdf_file)

In [ ]:
# f = open('/Users/Jon/Documents/columbia_syllabus/pdf/15-777_1b82.txt', 'r')
# sample_transcript = f.read()

In [ ]:
# sample_transcript